# Account & Position Discovery (TT-27 Epic)

Interactive exploration of the TastyTrade Account API — accounts, positions, balances, and transactions.
Used for discovery across all stories in the Account Management & Position Tracking epic.

In [1]:
import json
import os

import pandas as pd
from IPython.display import Markdown, display

from tastytrade.accounts import AccountsClient, Position
from tastytrade.config import RedisConfigManager
from tastytrade.connections import Credentials
from tastytrade.connections.requests import AsyncSessionHandler

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Set OBFUSCATE_ACCOUNTS=true in .env to mask account numbers in output
_OBFUSCATE = os.getenv("OBFUSCATE_ACCOUNTS", "false").lower() == "true"


def mask(account_number: str) -> str:
    """Mask account number for safe display, showing only last 4 chars."""
    if not _OBFUSCATE or len(account_number) <= 4:
        return account_number
    return "***" + account_number[-4:]


def mask_json(text: str, account_numbers: list[str]) -> str:
    """Mask all known account numbers in a raw JSON string."""
    if not _OBFUSCATE:
        return text
    for acct in account_numbers:
        text = text.replace(acct, mask(acct))
    return text


print(f"Account obfuscation: {'ON' if _OBFUSCATE else 'OFF'}")

Account obfuscation: ON


# Connect

In [2]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")
session = await AsyncSessionHandler.create(credentials)
client = AccountsClient(session)

print(f"Connected to {credentials.base_url}")

Connected to https://api.tastyworks.com


# Accounts

In [3]:
accounts = await client.get_accounts()

accounts_data = []
for a in accounts:
    accounts_data.append(
        {
            "Account": mask(a.account_number),
            "Type": a.account_type_name,
            "Margin/Cash": a.margin_or_cash,
            "Futures": a.is_futures_approved,
            "Objective": a.investment_objective,
            "Options Level": a.suitable_options_level,
            "Opened": a.opened_at,
            "Funded": a.funding_date,
        }
    )

df_accounts = pd.DataFrame(accounts_data)
display(Markdown(f"**{len(accounts)} accounts found**"))
display(df_accounts)

**2 accounts found**

,Account,Type,Margin/Cash,Futures,Objective,Options Level,Opened,Funded
0,***9822,Traditional IRA,Cash,True,SPECULATION,Defined Risk Spreads Plus Naked,2024-12-02 19:20:16.477000+00:00,2024-12-10
1,***1782,Individual,Margin,True,SPECULATION,No Restrictions,2021-10-02 06:03:33.654000+00:00,2023-05-09


# Balances

In [4]:
balances = []
for a in accounts:
    try:
        bal = await client.get_balances(a.account_number)
        balances.append(
            {
                "Account": mask(bal.account_number),
                "Cash": bal.cash_balance,
                "Net Liq": bal.net_liquidating_value,
                "Margin Equity": bal.margin_equity,
                "Equity BP": bal.equity_buying_power,
                "Derivative BP": bal.derivative_buying_power,
                "Day Trading BP": bal.day_trading_buying_power,
                "Maint Req": bal.maintenance_requirement,
                "Maint Excess": bal.maintenance_excess,
                "Long Equity": bal.long_equity_value,
                "Long Derivative": bal.long_derivative_value,
                "Long Futures": bal.long_futures_value,
                "Updated": bal.updated_at,
            }
        )
    except Exception as e:
        balances.append({"Account": mask(a.account_number), "Cash": f"Error: {e}"})

df_balances = pd.DataFrame(balances)
display(Markdown("**Account Balances**"))
display(df_balances)

**Account Balances**

,Account,Cash,Net Liq,Margin Equity,Equity BP,Derivative BP,Day Trading BP,Maint Req,Maint Excess,Long Equity,Long Derivative,Long Futures,Updated
0,***9822,39249.50,176149.80,178821.80,24872.48,24872.48,0.0,153336.64,24872.48,139572.3,114.5,0.0,2026-02-09 15:37:33.577000+00:00
1,***1782,1446.36,1446.36,1446.36,1446.36,1446.36,0.0,0.00,1446.36,0.0,0.0,0.0,2025-12-17 10:33:27.043000+00:00


# Positions

In [5]:
all_positions: list[Position] = []
for a in accounts:
    positions = await client.get_positions(a.account_number)
    all_positions.extend(positions)
    print(
        f"{mask(a.account_number)} ({a.account_type_name}): {len(positions)} positions"
    )

if all_positions:
    positions_data = []
    for p in all_positions:
        positions_data.append(
            {
                "Account": mask(p.account_number),
                "Symbol": p.symbol,
                "Underlying": p.underlying_symbol,
                "Type": p.instrument_type.value,
                "Qty": p.quantity,
                "Direction": p.quantity_direction.value,
                "Avg Open": p.average_open_price,
                "Mark": p.mark_price or p.mark,
                "Close": p.close_price,
                "Multiplier": p.multiplier,
                "Streamer": p.streamer_symbol,
                "Expires": p.expires_at,
                "Updated": p.updated_at,
            }
        )

    df_positions = pd.DataFrame(positions_data)
    display(
        Markdown(
            f"**{len(all_positions)} total positions across {len(accounts)} accounts**"
        )
    )
    display(df_positions)
else:
    display(Markdown("*No positions found*"))

***9822 (Traditional IRA): 12 positions
***1782 (Individual): 0 positions


**12 total positions across 2 accounts**

,Account,Symbol,Underlying,Type,Qty,Direction,Avg Open,Mark,Close,Multiplier,Streamer,Expires,Updated
0,***9822,./MESM6EX3H6 260320P6450,/MESM6,Future Option,3.00,Short,47.58,None,34.50,5.0,./EX3H26P6450:XCME,2026-03-20 20:00:00+00:00,2026-02-05 02:59:53.227000+00:00
1,***9822,./MCLJ6MCOJ6 260317C84,/MCLJ6,Future Option,1.00,Short,1.08,None,0.99,100.0,./MCOJ26C84:XNYM,2026-03-17 18:30:00+00:00,2026-02-06 18:13:13.133000+00:00
2,***9822,./MCLJ6MCOJ6 260317P55,/MCLJ6,Future Option,1.00,Short,0.59,None,0.64,100.0,./MCOJ26P55:XNYM,2026-03-17 18:30:00+00:00,2026-02-06 18:13:13.165000+00:00
3,***9822,MCD 260320P00305000,MCD,Equity Option,1.00,Short,2.74,None,2.31,100.0,.MCD260320P305,2026-03-20 20:00:00+00:00,2026-02-06 15:23:30.936000+00:00
4,***9822,MCD 260320P00295000,MCD,Equity Option,1.00,Long,1.54,None,1.24,100.0,.MCD260320P295,2026-03-20 20:00:00+00:00,2026-02-06 15:23:30.956000+00:00
5,***9822,./MESM6EX3H6 260320C7275,/MESM6,Future Option,3.00,Short,22.42,None,25.25,5.0,./EX3H26C7275:XCME,2026-03-20 20:00:00+00:00,2026-02-05 02:59:53.213000+00:00
6,***9822,./6EM6 EUUJ6 260403C1.225,/6EM6,Future Option,1.00,Short,0.00,None,0.00,125000.0,./EUUJ26C1.225:XCME,2026-04-03 14:00:00+00:00,2026-02-04 13:38:54.682000+00:00
7,***9822,./6EM6 EUUJ6 260403P1.16,/6EM6,Future Option,1.00,Short,0.00,None,0.00,125000.0,./EUUJ26P1.16:XCME,2026-04-03 14:00:00+00:00,2026-02-04 13:38:54.628000+00:00
8,***9822,CSCO 260227C00078000,CSCO,Equity Option,1.00,Short,2.00,None,7.98,100.0,.CSCO260227C78,2026-02-27 21:00:00+00:00,2026-01-15 16:39:59.871000+00:00
9,***9822,SPY,SPY,Equity,100.29,Long,664.93,None,690.62,1.0,SPY,NaT,2026-01-31 10:33:24.603000+00:00


## Positions by Instrument Type

In [6]:
if all_positions:
    type_summary = (
        df_positions.groupby("Type")
        .agg(
            Count=("Symbol", "count"),
            Symbols=("Symbol", lambda x: ", ".join(sorted(set(x)))),
        )
        .sort_values("Count", ascending=False)
    )
    display(Markdown("**Position breakdown by instrument type**"))
    display(type_summary)

    # Streamer symbol coverage
    has_streamer = sum(1 for p in all_positions if p.streamer_symbol)
    display(
        Markdown(
            f"**Streamer symbol coverage:** {has_streamer}/{len(all_positions)} positions have DXLink streamer symbols"
        )
    )

**Position breakdown by instrument type**

,Count,Symbols
Type,,
Future Option,6,"./6EM6 EUUJ6 260403C1.225, ./6EM6 EUUJ6 260403P1.16, ./MCLJ6MCOJ6 260317C84, ./MCLJ6MCOJ6 260317P55, ./MESM6EX3H6 260320C7275, ./MESM6EX3H6 260320P6450"
Equity,3,"CSCO, QQQ, SPY"
Equity Option,3,"CSCO 260227C00078000, MCD 260320P00295000, MCD 260320P00305000"


**Streamer symbol coverage:** 11/12 positions have DXLink streamer symbols

# Transactions (Recent)

In [7]:
# Transactions use raw API — not yet modeled in the SDK
acct = accounts[0].account_number if accounts else None

if acct:
    async with session.session.get(
        f"{session.base_url}/accounts/{acct}/transactions", params={"per-page": 20}
    ) as resp:
        data = await resp.json()
        txns = data.get("data", {}).get("items", [])

    if txns:
        txn_data = []
        for t in txns:
            txn_data.append(
                {
                    "Date": t.get("executed-at", t.get("transaction-date", "")),
                    "Action": t.get("action", ""),
                    "Symbol": t.get("underlying-symbol", t.get("symbol", "")),
                    "Instrument": t.get("instrument-type", ""),
                    "Description": t.get("description", ""),
                    "Value": t.get("value", ""),
                    "Effect": t.get("value-effect", ""),
                    "Commission": t.get("commission", ""),
                    "Fees": t.get("clearing-fees", ""),
                }
            )
        df_txns = pd.DataFrame(txn_data)
        display(Markdown(f"**Last {len(txns)} transactions for {mask(acct)}**"))
        display(df_txns)
    else:
        display(Markdown("*No transactions found*"))

**Last 20 transactions for ***9822**

,Date,Action,Symbol,Instrument,Description,Value,Effect,Commission,Fees
0,2026-02-07T17:11:17.847Z,,,,Regulatory fee adjustment,0.007,Debit,,
1,2026-02-06T18:13:11.536Z,Sell to Open,/MCLJ6,Future Option,Sold 1 /MCLJ6 MCOJ6 03/17/26 Call 84.00 @ 1.08,108.0,Credit,0.75,0.3
2,2026-02-06T18:13:11.536Z,Sell to Open,/MCLJ6,Future Option,Sold 1 /MCLJ6 MCOJ6 03/17/26 Put 55.00 @ 0.59,59.0,Credit,0.75,0.3
3,2026-02-06T15:23:28.947Z,Sell to Open,MCD,Equity Option,Sold 1 MCD 03/20/26 Put 305.00 @ 2.74,274.0,Credit,1.0,0.1
4,2026-02-06T15:23:28.947Z,Buy to Open,MCD,Equity Option,Bought 1 MCD 03/20/26 Put 295.00 @ 1.54,154.0,Debit,1.0,0.1
5,2026-02-05T02:59:51.269Z,Sell to Open,/MESM6,Future Option,Sold 1 /MESM6 EX3H6 03/20/26 Call 7275.00 @ 22.25,111.25,Credit,0.75,0.3
6,2026-02-05T02:59:51.269Z,Sell to Open,/MESM6,Future Option,Sold 1 /MESM6 EX3H6 03/20/26 Put 6450.00 @ 47.75,238.75,Credit,0.75,0.3
7,2026-02-05T02:58:54.087Z,Sell to Open,/MESM6,Future Option,Sold 2 /MESM6 EX3H6 03/20/26 Call 7275.00 @ 22.5,225.0,Credit,1.5,0.6
8,2026-02-05T02:58:54.087Z,Sell to Open,/MESM6,Future Option,Sold 2 /MESM6 EX3H6 03/20/26 Put 6450.00 @ 47.5,475.0,Credit,1.5,0.6
9,2026-02-04T13:38:51.301Z,Sell to Open,/6EM6,Future Option,Sold 1 /6EM6 EUUJ6 04/03/26 Call 1.22500 @ 0.0032,400.0,Credit,1.25,0.3


# Trading Status

In [8]:
for a in accounts:
    async with session.session.get(
        f"{session.base_url}/accounts/{a.account_number}/trading-status"
    ) as resp:
        data = await resp.json()
        status = data.get("data", {})

    display(Markdown(f"### {mask(a.account_number)} ({a.account_type_name})"))
    status_items = [
        ("Day Trade Count", status.get("day-trade-count", "N/A")),
        ("Equities Margin Calc", status.get("equities-margin-calculation-type", "N/A")),
        ("Options Level", status.get("options-level", "N/A")),
        ("Short Calls", status.get("short-calls-enabled", "N/A")),
        ("Naked Options", status.get("are-naked-options-enabled", "N/A")),
        ("Futures Enabled", status.get("are-futures-enabled", "N/A")),
        ("Crypto Enabled", status.get("is-cryptocurrency-enabled", "N/A")),
        ("Max Order Size", status.get("max-symbol-quantity", "N/A")),
    ]
    df_status = pd.DataFrame(status_items, columns=["Setting", "Value"])
    display(df_status)

### ***9822 (Traditional IRA)

,Setting,Value
0,Day Trade Count,0
1,Equities Margin Calc,IRA Margin
2,Options Level,Defined Risk Spreads Plus Naked
3,Short Calls,True
4,Naked Options,N/A
5,Futures Enabled,N/A
6,Crypto Enabled,False
7,Max Order Size,N/A


### ***1782 (Individual)

,Setting,Value
0,Day Trade Count,0
1,Equities Margin Calc,Reg T
2,Options Level,No Restrictions
3,Short Calls,True
4,Naked Options,N/A
5,Futures Enabled,N/A
6,Crypto Enabled,True
7,Max Order Size,N/A


# API Response Explorer

Raw API inspection — use this to discover new fields and response shapes.

In [9]:
# Change the endpoint to explore any API path
acct = accounts[0].account_number if accounts else "REPLACE_ME"
endpoint = f"/accounts/{acct}/positions"

async with session.session.get(f"{session.base_url}{endpoint}") as resp:
    data = await resp.json()
    raw = json.dumps(data, indent=2, default=str)[:5000]
    all_acct_nums = [a.account_number for a in accounts]
    print(mask_json(raw, all_acct_nums))

{
  "data": {
    "items": [
      {
        "account-number": "***9822",
        "instrument-type": "Future Option",
        "streamer-symbol": "./EX3H26P6450:XCME",
        "symbol": "./MESM6EX3H6 260320P6450",
        "underlying-symbol": "/MESM6",
        "quantity": 3,
        "average-daily-market-close-price": "34.5",
        "average-open-price": "47.58333333",
        "average-yearly-market-close-price": "47.58333333",
        "close-price": "34.5",
        "cost-effect": "Debit",
        "is-frozen": false,
        "is-suppressed": false,
        "multiplier": 5,
        "quantity-direction": "Short",
        "restricted-quantity": 0,
        "expires-at": "2026-03-20T20:00:00.000+00:00",
        "realized-day-gain": "0.0",
        "realized-day-gain-date": "2026-02-05",
        "realized-day-gain-effect": "None",
        "realized-today": "0.0",
        "realized-today-date": "2026-02-05",
        "realized-today-effect": "None",
        "created-at": "2026-02-05T02:58:54.22

# Cleanup

In [10]:
await session.close()
print("Session closed.")

Session closed.
